In [31]:
import chess_game_tracker

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

tokens_filepath = "C:\\Users\\anton\\Documents\\GitHub\\chess-gpt\\data\\tokens.txt"
intermediate_filepath = "C:\\Users\\anton\\Documents\\GitHub\\chess-gpt\\data\\intermediate_non_bullet.txt"
model_filepath = "C:\\Users\\anton\\Documents\\GitHub\\chess-gpt\\data\\model_2014_non_bullet"



# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
eval_interval = 200
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 400
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

print (device)

cuda


In [4]:
with open(tokens_filepath, "r") as f:
    tokens = f.read().splitlines()
    
tokens = ["ZERO", "START"] + tokens

token_to_int = {}
int_to_token = {}

for i, token in enumerate(tokens):
    token_to_int[token] = i
    int_to_token[i] = token
    
def encode(lst):
    return [token_to_int[token] for token in lst]
def decode(lst):
    return [int_to_token[i] for i in lst]

vocab_size = len(tokens)
print (vocab_size)

360


In [5]:
encoded_games = []
with open(intermediate_filepath, "r") as f:
    for line in f:
        game = encode(["START"] + line.split())
        if len(game) >= block_size + 1:
            encoded_games.append(game)

In [6]:
print (len(encoded_games))
print (len(encoded_games[100]))
print (len(encoded_games[200]))
print (encoded_games[300])
print (min([len(g) for g in encoded_games]))

33206
143
73
[1, 169, 145, 81, 80, 148, 184, 208, 164, 189, 84, 212, 238, 195, 180, 213, 21, 228, 49, 63, 104, 282, 262, 23, 332, 346, 225, 231, 235, 19, 218, 59, 342, 351, 349, 332, 89, 81, 271, 291, 84, 83, 24, 295, 70, 302, 303, 298, 298, 89, 76, 69, 76, 67, 81, 128, 81, 87, 82, 87, 68, 91, 125, 152, 137, 15, 80, 8, 72, 86, 74, 85, 325, 4, 44, 7, 51, 67, 312, 16, 72, 91, 72, 328, 66, 12, 330, 322, 74, 198, 102, 25, 67, 29, 118, 130, 99, 84, 48, 86, 115, 84, 81, 89, 78, 88, 78, 107, 66, 74, 66, 33, 66, 29, 43, 33, 70, 29, 74, 33, 74, 29, 74, 33, 71, 29, 74, 70, 66, 71]
65


In [7]:
n = int(0.9*len(encoded_games)) # first 90% will be train, rest val
train_data = encoded_games[:n]
val_data = encoded_games[n:]

In [8]:
def get_block_from_game(game, block_sz, rnd, offset):
    i = rnd % (len(game) - block_sz)
    t = game[i + offset : i + block_sz + offset]
    if len(t) != block_sz:
        print (game)
        print (t)
        print (i, offset, block_sz)
        assert False
    return torch.tensor(t, dtype=torch.long)

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-1, (batch_size, 2))
    x = torch.stack([get_block_from_game(encoded_games[i[0]], block_size, i[1], 0) for i in ix])
    y = torch.stack([get_block_from_game(encoded_games[i[0]], block_size, i[1], 1) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

In [16]:
def generate(model, idx, max_new_tokens, game):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)

        legal_moves = encode(game.encoded_legal_moves())
#         print (legal_moves)
        if len(legal_moves) == 0:
            return idx

        for i in range(vocab_size):
            if i not in legal_moves:
                logits[0, i] = -100000
        
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        
#         print (idx_next,  idx_next.tolist()[0], "hi")
        assert idx_next.tolist()[0][0] in legal_moves
        
        game.make_move(decode(idx_next.tolist()[0])[0])
        
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

In [22]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')



8.01076 M parameters


In [28]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [29]:
for iter in range(10000000 + 1):
# while True:
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    if iter % (eval_interval * 10) == 0:
        print("saving")
        torch.save(m, model_filepath + ".pt")
        torch.save(m.state_dict(), model_filepath + "_state.pt")
        
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.1983, val loss 2.1795
saving
step 200: train loss 2.0467, val loss 2.0081
step 400: train loss 1.9899, val loss 1.9546
step 600: train loss 1.9503, val loss 1.8977
step 800: train loss 1.9229, val loss 1.9051
step 1000: train loss 1.8984, val loss 1.8627
step 1200: train loss 1.8568, val loss 1.8577
step 1400: train loss 1.8626, val loss 1.8331
step 1600: train loss 1.8381, val loss 1.8275
step 1800: train loss 1.8398, val loss 1.7935
step 2000: train loss 1.8423, val loss 1.7659
saving
step 2200: train loss 1.8122, val loss 1.7684
step 2400: train loss 1.8001, val loss 1.7837
step 2600: train loss 1.7892, val loss 1.7454
step 2800: train loss 1.7684, val loss 1.7429
step 3000: train loss 1.7561, val loss 1.7383
step 3200: train loss 1.7645, val loss 1.7326
step 3400: train loss 1.7451, val loss 1.7067
step 3600: train loss 1.7583, val loss 1.6972
step 3800: train loss 1.7336, val loss 1.7075
step 4000: train loss 1.7300, val loss 1.6875
saving
step 4200: train los

step 34600: train loss 1.1886, val loss 1.1431
step 34800: train loss 1.1759, val loss 1.1462
step 35000: train loss 1.1690, val loss 1.1457
step 35200: train loss 1.1731, val loss 1.1497
step 35400: train loss 1.1941, val loss 1.1458
step 35600: train loss 1.1861, val loss 1.1398
step 35800: train loss 1.1629, val loss 1.1298
step 36000: train loss 1.1806, val loss 1.1254
saving
step 36200: train loss 1.1648, val loss 1.1409
step 36400: train loss 1.1670, val loss 1.1213
step 36600: train loss 1.1617, val loss 1.1400
step 36800: train loss 1.1775, val loss 1.1389
step 37000: train loss 1.1517, val loss 1.1131
step 37200: train loss 1.1526, val loss 1.1039
step 37400: train loss 1.1581, val loss 1.1117
step 37600: train loss 1.1640, val loss 1.1213
step 37800: train loss 1.1524, val loss 1.0948
step 38000: train loss 1.1611, val loss 1.1173
saving
step 38200: train loss 1.1620, val loss 1.1132
step 38400: train loss 1.1526, val loss 1.1023
step 38600: train loss 1.1493, val loss 1.1058

step 69000: train loss 0.9601, val loss 0.9276
step 69200: train loss 0.9630, val loss 0.9217
step 69400: train loss 0.9640, val loss 0.9466
step 69600: train loss 0.9573, val loss 0.9066
step 69800: train loss 0.9430, val loss 0.9080
step 70000: train loss 0.9553, val loss 0.9117
saving
step 70200: train loss 0.9524, val loss 0.9184
step 70400: train loss 0.9599, val loss 0.9150
step 70600: train loss 0.9605, val loss 0.9177
step 70800: train loss 0.9573, val loss 0.9444
step 71000: train loss 0.9591, val loss 0.9438
step 71200: train loss 0.9342, val loss 0.9104
step 71400: train loss 0.9620, val loss 0.9175
step 71600: train loss 0.9475, val loss 0.9098
step 71800: train loss 0.9433, val loss 0.9095
step 72000: train loss 0.9584, val loss 0.8942
saving
step 72200: train loss 0.9413, val loss 0.9097
step 72400: train loss 0.9598, val loss 0.9141
step 72600: train loss 0.9403, val loss 0.9240
step 72800: train loss 0.9366, val loss 0.9239
step 73000: train loss 0.9444, val loss 0.9078

step 103400: train loss 0.8517, val loss 0.8337
step 103600: train loss 0.8548, val loss 0.8173
step 103800: train loss 0.8453, val loss 0.8131
step 104000: train loss 0.8308, val loss 0.8121
saving
step 104200: train loss 0.8615, val loss 0.8101
step 104400: train loss 0.8557, val loss 0.8292
step 104600: train loss 0.8440, val loss 0.8082
step 104800: train loss 0.8414, val loss 0.8100
step 105000: train loss 0.8451, val loss 0.8094
step 105200: train loss 0.8417, val loss 0.8139
step 105400: train loss 0.8568, val loss 0.8060
step 105600: train loss 0.8422, val loss 0.8118
step 105800: train loss 0.8379, val loss 0.8094
step 106000: train loss 0.8557, val loss 0.8220
saving
step 106200: train loss 0.8463, val loss 0.8131
step 106400: train loss 0.8442, val loss 0.8015
step 106600: train loss 0.8378, val loss 0.8087
step 106800: train loss 0.8258, val loss 0.7965
step 107000: train loss 0.8355, val loss 0.7970
step 107200: train loss 0.8335, val loss 0.8163
step 107400: train loss 0.

KeyboardInterrupt: 

In [33]:
game = chess_game_tracker.Game()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
context[0][0] = 1
print(decode(generate(m, idx = context, max_new_tokens=100, game=game)[0].tolist()))
print (game.get_game_pgn())

['START', 'Pawn:d2:TwoPush', 'Pawn:e7:Push', 'Pawn:e2:TwoPush', 'Pawn:d7:TwoPush', 'Pawn:e4:Push', 'Pawn:c7:Push', 'Pawn:f2:TwoPush', 'Pawn:f7:TwoPush', 'Pawn:c2:Push', 'Pawn:b7:Push', 'N2:-1:2', 'N2:-2:-1', 'BD:e3', 'Pawn:g7:Push', 'N1:2:1', 'BD:g7', 'Pawn:h2:Push', 'BL:a6', 'BL:d3', 'K:2:0', 'Q:c2', 'BL:d3', 'Q:d3', 'N1:2:-1', 'K:-2:0', 'Pawn:c6:Push', 'Pawn:g2:TwoPush', 'R1:2:0', 'K:-1:0', 'Pawn:b6:Push', 'Q:e2', 'Pawn:c5:Push', 'Pawn:g4:Push', 'Pawn:b5:Push', 'Pawn:c3:CaptureLeft', 'N2:-2:-1', 'Pawn:h3:Push', 'N2:-1:-2', 'K:-1:0', 'Pawn:c4:Push', 'Pawn:b2:CaptureRight', 'N2:-1:-2', 'K:1:1', 'N2:2:-1', 'R2:-3:0', 'Pawn:h7:TwoPush', 'Pawn:g5:CaptureRight', 'K:1:-1', 'Pawn:h6:CaptureLeft', 'K:-1:0', 'Pawn:h4:Push', 'K:-1:0', 'N2:1:2', 'K:-1:1', 'N2:-2:-1', 'Q:a5', 'N2:-1:2', 'K:-1:0', 'N2:2:-1', 'Q:b5', 'Q:b5', 'R1:0:-3', 'N2:-1:2', 'K:1:-1', 'R2:1:0', 'K:-1:1', 'Pawn:h5:Push', 'K:1:-1', 'Pawn:h6:Push', 'R2:2:0', 'K:-1:1', 'K:-1:1', 'K:0:1', 'K:-1:-1', 'K:0:1', 'K:1:1', 'K:0:1', 'K:1:

In [100]:
torch.save(m, model_filepath + ".pt")
torch.save(m.state_dict(), model_filepath + "_state.pt")